# Al Tableau & PHREEQC

In [9]:
% setup environment
% set path to equilibrium solver
txt=pwd;
n=length(txt);
rootpath=txt; rootpath=txt(1:n-19);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
%graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [10]:
% define equilibrium problem

AlT=1e-2;  pH=7; pe=20.75-pH; T=25; show=0;


In [11]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=2; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=1; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
%database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

%for i=1:length(pH)

TOTALS=[AlT]';
TOTALS(TOTALS==0)=1e-16;% get rid of zero values, b/c div by zero error
%TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;

tic;
[Al,AlOH3,MASSERR]=Altableau(pH,pe,T,flag1,flag2,flag3,flag4,flag5);
tableautime=toc;
Almasserror=MASSERR(1);


In [15]:
% MODEL USING PHREEQC

minerals=[{'Gibbsite'}]; totalvector=[AlT; 3*AlT]; totalnames=[{'Al'}; {'Cl'}]; 
speciesexport=[{'Al+3'}; {'OH-'};{'AlOH+2'};{'Al(OH)2+'};{'Al2(OH)2+4'};{'Al3(OH)4+5'};{'Al13O4(OH)24+7'}];
acid=['HCl']; 
database=['Al_species.dat']; 

tic;
[solutionspeciesconcs, speciesnames, solidconcs, solidnames]=runPHREEQCv2noHA(T,pH,pe,totalnames,totalvector,minerals,speciesexport,database,show,acid);
PHREEQCtime=toc;

Alw13PHREEQC=solutionspeciesconcs(1); 
OHPHREEQC=solutionspeciesconcs(2); 
AlOHPHREEQC=solutionspeciesconcs(3); 
AlOH2PHREEQC=solutionspeciesconcs(4); 
Al2OH2PHREEQC=solutionspeciesconcs(5); 
Al3OH4PHREEQC=solutionspeciesconcs(6);
Al13PHREEQC=solutionspeciesconcs(7);
AlOH3sw13PHREEQC=solidconcs(1);

AlmasserrorPHREEQC=AlT-AlOHPHREEQC-AlOH2PHREEQC-2*Al2OH2PHREEQC-3*Al3OH4PHREEQC-13*Al13PHREEQC-AlOH3sw13PHREEQC;
end

Execution exception: parse error:

  syntax error




In [21]:
Alcompare=[Al Alw13PHREEQC]
AlOH3scompare=[AlOH3 AlOH3sw13PHREEQC]
Almasserrorcompare=[Almasserror AlmasserrorPHREEQC]
timecompare=[tableautime PHREEQCtime]

Alcompare,1,2
1,1.75388e-29,1.57774e-13


AlOH3scompare,1,2
1,25,0.01


Almasserrorcompare,1,2
1,0,1.2737e-13


timecompare,1,2
1,0.128939,0.0217979


fatal: caught signal Aborted -- stopping myself...